The idea is simple: for a crossing just enter `('x',(t,l,r))`, where `t`, `l`, `r` are segments on the top, on the left, and on the right of the top. (This is akin to a PD-code but different, since Wirtinger calculations use a different labeling) For a vertex just enter `('v',((vi1,vi2),(vo1,vo2))` and so on -- first incoming, then outgoing in counterclockwisee order.

Please dont label edge segments with jumps :)

In [1]:
from subprocess import check_output
from sage.misc.parser import Parser
p = Parser(make_var=var)

In [2]:
def get_relation(x):
    if x[0] == 'x':
        t, l, r = x[1]
        if l-r==1:
            return [r, t, -l, -t]
        else:
            return [t, l, -t, -r]
        
    if x[0] == 'v':
        vin, vout = x[1]
        return [v for v in vin] + [-v for v in vout]
        
def D(r, i, xs):
    x = xs[r[0]-1] if r[0] > 0 else (xs[-r[0]-1])**(-1)
    if len(r) == 1:
        if r[0] > 0:
            if r[0] == i:
                return 1
            else:
                return 0
        else:
            if -r[0] == i:
                return -x
            else:
                return 0
    else:
        return D([r[0]],i, xs)+ x*D(r[1:], i, xs)
    
def get_matrix(G):
    relations = list(map(get_relation, G))
    n_var = max([abs(x) for x in sum(relations, [])])
    xs = [var('x'+str(i)) for i in range(1, n_var+1)]
    
    l = [i for i in range(n_var)]
    
    def find(seg):
        if seg == l[seg]:
            return seg
        l[seg] = find(l[seg])
        return l[seg]

    def union(seg_i, seg_j):
        iloc = find(seg_i)
        jloc = find(seg_j)

        if iloc <= jloc:
            l[jloc] = iloc
        else:
            l[iloc] = jloc
        return 0
    
    for _, r in [rel for rel in G if rel[0] == 'x']:
        union(r[1]-1,r[2]-1)
    
    union_result = [find(i) for i in range(n_var)]
    
    n_edge = len(set(union_result))

    ts = {label:var('t'+str(i+1)) for i, label in enumerate(list(set(union_result))) }
    homomorphism = {x:ts[union_result[i]] for i, x in enumerate(xs)}
    
    print('found: '+ str(n_var)+' segments and '+str(n_edge)+' edges')
    
    print('the homomorphism from pi_1(X) into an abelian group generated by edges is:')
    print(homomorphism)
    
    M = [[0 for j in range(len(relations))] for i in range(n_var)]
    for i in range(n_var):
        for j in range(len(relations)):
            entry = D(relations[j], i+1, xs)
            M[i][j] = entry(homomorphism)
    return matrix(M)

def get_minors(M):
    print("got matrix of size "+str((M.nrows(),M.ncols())))
    print('computing minors...')
    mins = M.minors(M.ncols())
    i = 0

    while all([a == 0 for a in mins]) and i < M.ncols()-1:
        i += 1
        print('trying order '+str(M.ncols()-i)+'...')
        mins = M.minors(M.ncols()-i)
    return [m for m in mins if m != 0]

def get_GCD(minors):
    print('running wolframscript with '+str(len(minors))+' minors...')
    return p.parse(check_output('wolframscript -c "PolynomialGCD['+', '.join([str(m) for m in minors])+ \
                        ']//Simplify//Expand"', shell = True).strip().decode("utf-8"))

def get_Alexander_poly(G):
    return get_GCD(get_minors(get_matrix(G)))

def get_poly_wolfram(G):
    M = get_matrix(G)
    print("got matrix of size "+str((M.nrows(),M.ncols())))
    print('computing minors...')
    order = M.ncols()
    M_str = '{'+', '.join(['{'+', '.join([str(el) for el in list(row)])+'}' for row in list(M)])+'}'
    command = '"PolynomialGCD@@Flatten[Minors['+M_str+', '+str(order)+']]"'
    res = check_output('wolframscript -c '+command, shell = True).strip().decode("utf-8")
    print('got GCD '+res+'...')

    while res == '0' and order > 1:
        order -= 1
        print('trying order '+str(order)+'...')
        command = '"PolynomialGCD@@Flatten[Minors['+M_str+', '+str(order)+']]"'
        res = check_output('wolframscript -c '+command, shell = True).strip().decode("utf-8")
        print('got GCD '+res+'...')
    
    return p.parse(res)

In [3]:
#example 1: trivial stevedore theta
G = [
    ('v', [[8],[9,1]]),
    ('v', [[7,9],[8]]),
    ('x', [7,5,6]),
    ('x', [5,1,2]),
    ('x', [2,4,5]),
    ('x', [4,2,3]),
    ('x', [6,4,3]),
    ('x', [3,7,6])
]
get_poly_wolfram(G)

found: 9 segments and 3 edges
the homomorphism from pi_1(X) into an abelian group generated by edges is:
{x1: t1, x2: t1, x3: t1, x4: t1, x5: t1, x6: t1, x7: t1, x8: t3, x9: t2}
got matrix of size (9, 8)
computing minors...
got GCD 0...
trying order 7...
got GCD 2/(t2*t3) - (5*t1)/(t2*t3) + (2*t1^2)/(t2*t3)...


2*t1^2/(t2*t3) - 5*t1/(t2*t3) + 2/(t2*t3)

In [4]:
# see goodnotes: wrap around one edge
G = [
    ('v', [[4,1],[5]]),
    ('v', [[9],[1,2]]),
    ('x', [4,7,6]),
    ('x', [6,4,3]),
    ('x', [7,6,5]),
    ('x', [7,8,9]),
    ('x', [2,7,8]),
    ('x', [8,2,3]),
]

A = get_GCD(get_minors(get_matrix(G)))
A, get_poly_wolfram(G)

found: 9 segments and 3 edges
the homomorphism from pi_1(X) into an abelian group generated by edges is:
{x1: t1, x2: t2, x3: t2, x4: t2, x5: t3, x6: t3, x7: t3, x8: t3, x9: t3}
got matrix of size (9, 8)
computing minors...
trying order 7...
running wolframscript with 264 minors...
found: 9 segments and 3 edges
the homomorphism from pi_1(X) into an abelian group generated by edges is:
{x1: t1, x2: t2, x3: t2, x4: t2, x5: t3, x6: t3, x7: t3, x8: t3, x9: t3}
got matrix of size (9, 8)
computing minors...
got GCD 0...
trying order 7...
got GCD 1/(t1^9*t2^2*t3)...


(1/(t1*t2*t3), 1/(t1^9*t2^2*t3))

In [5]:
# see goodnotes: wrap arpund two edges
G = [
    ('v', [[3,8],[9]]),
    ('v', [[15],[4,1]]),
    ('x', [6,8,7]),
    ('x', [6,4,5]),
    ('x', [7,10,9]),
    ('x', [9,7,6]),
    ('x', [5,10,11]),
    ('x', [11,5,6]),
    ('x', [13,12,11]),
    ('x', [13,14,15]),
    ('x', [3,13,12]),
    ('x', [12,3,2]),
    ('x', [14,1,2]),
    ('x', [1,13,14])
]

get_poly_wolfram(G)

found: 15 segments and 3 edges
the homomorphism from pi_1(X) into an abelian group generated by edges is:
{x1: t1, x2: t1, x3: t1, x4: t3, x5: t3, x6: t3, x7: t3, x8: t3, x9: t2, x10: t2, x11: t2, x12: t2, x13: t2, x14: t2, x15: t2}
got matrix of size (15, 14)
computing minors...
got GCD 0...
trying order 13...
got GCD 1/(t1^22*t2*t3^2)...


1/(t1^22*t2*t3^2)

In [6]:
# see goodnotes: wrap around three edges, symmetrical
G = [
    ('v', [[21,14],[1]]),
    ('v', [[7],[8,15]]),
    ('x', [3,1,2]),
    ('x', [3,5,4]),
    ('x', [8,4,3]),
    ('x', [4,9,8]),
    ('x', [2,9,10]),
    ('x', [10,2,3]),
    ('x', [5,19,20]),
    ('x', [20,5,6]),
    ('x', [18,7,6]),
    ('x', [7,19,18]),
    ('x', [19,18,17]),
    ('x', [19,20,21]),
    ('x', [17,13,12]),
    ('x', [13,17,16]),
    ('x', [11,15,16]),
    ('x', [15,11,12]),
    ('x', [12,11,10]),
    ('x', [12,13,14])
]

get_poly_wolfram(G)

found: 21 segments and 3 edges
the homomorphism from pi_1(X) into an abelian group generated by edges is:
{x1: t1, x2: t1, x3: t1, x4: t1, x5: t1, x6: t1, x7: t1, x8: t3, x9: t3, x10: t3, x11: t3, x12: t3, x13: t3, x14: t3, x15: t2, x16: t2, x17: t2, x18: t2, x19: t2, x20: t2, x21: t2}
got matrix of size (21, 20)
computing minors...
got GCD 0...
trying order 19...
got GCD 1/(t1^16*t2^77*t3^389)...


1/(t1^16*t2^77*t3^389)

In [7]:
G = [
    ('v', [[9,4],[10]]),
    ('v', [[14],[5,1]]),
    ('x', [1,13,12]),
    ('x', [12,2,1]),
    ('x', [2,12,11]),
    ('x', [5,11,10]),
    ('x', [5,2,3]),
    ('x', [3,5,6]),
    ('x', [6,3,4]),
    ('x', [14,8,9]),
    ('x', [14,7,6]),
    ('x', [7,14,13]),
    ('x', [13,8,7]),
]

get_poly_wolfram(G)

found: 14 segments and 3 edges
the homomorphism from pi_1(X) into an abelian group generated by edges is:
{x1: t1, x2: t1, x3: t1, x4: t1, x5: t3, x6: t3, x7: t3, x8: t3, x9: t3, x10: t2, x11: t2, x12: t2, x13: t2, x14: t2}
got matrix of size (14, 13)
computing minors...
got GCD 1/(t1*t2*t3)...


1/(t1*t2*t3)

In [8]:
G = [
    ('v', [[9],[1,6]]),
    ('v', [[2,5],[3]]),
    ('x', [7,4,3]),
    ('x', [3,7,6]),
    ('x', [4,2,1]),
    ('x', [8,4,5]),
    ('x', [5,8,9]),
    ('x', [1,7,8]),
]

get_poly_wolfram(G)

found: 9 segments and 3 edges
the homomorphism from pi_1(X) into an abelian group generated by edges is:
{x1: t1, x2: t1, x3: t2, x4: t2, x5: t2, x6: t3, x7: t3, x8: t3, x9: t3}
got matrix of size (9, 8)
computing minors...
got GCD 1 - t1^(-1)...


-1/t1 + 1

In [9]:
from slice_thetas import *


res = {k:get_poly_wolfram(g) for k, g in slice_thetas.items()}

print('\n\n results')

for k, p in res.items():
    print(k, p)

found: 8 segments and 3 edges
the homomorphism from pi_1(X) into an abelian group generated by edges is:
{x1: t1, x2: t1, x3: t2, x4: t2, x5: t2, x6: t3, x7: t3, x8: t3}
got matrix of size (8, 7)
computing minors...
got GCD 1/(t1*t2)...
found: 9 segments and 3 edges
the homomorphism from pi_1(X) into an abelian group generated by edges is:
{x1: t1, x2: t1, x3: t2, x4: t2, x5: t2, x6: t3, x7: t3, x8: t3, x9: t3}
got matrix of size (9, 8)
computing minors...
got GCD 1/(t1*t2)...
found: 10 segments and 3 edges
the homomorphism from pi_1(X) into an abelian group generated by edges is:
{x1: t1, x2: t1, x3: t1, x4: t2, x5: t2, x6: t2, x7: t2, x8: t3, x9: t3, x10: t3}
got matrix of size (10, 9)
computing minors...
got GCD 1/(t1*t2*t3)...
found: 10 segments and 3 edges
the homomorphism from pi_1(X) into an abelian group generated by edges is:
{x1: t1, x2: t1, x3: t2, x4: t2, x5: t2, x6: t2, x7: t3, x8: t3, x9: t3, x10: t3}
got matrix of size (10, 9)
computing minors...
got GCD 1/(t1*t3)...
fou

In [3]:
get_poly_wolfram([
 ('v', [[1],[2,4]]),
 ('v', [[3,6],[1]]),
 ('x', [3,4,5]),
 ('x', [5,2,3]),
 ('x', [2,5,6])
])

found: 6 segments and 3 edges
the homomorphism from pi_1(X) into an abelian group generated by edges is:
{x1: t1, x2: t2, x3: t2, x4: t3, x5: t3, x6: t3}
got matrix of size (6, 5)
computing minors...
got GCD t2^(-1) + t2/t3...


t2/t3 + 1/t2

In [5]:
get_poly_wolfram([('v', [[1],[2,3]]),
 ('v', [[2,3],[1]])])

found: 3 segments and 3 edges
the homomorphism from pi_1(X) into an abelian group generated by edges is:
{x1: t1, x2: t2, x3: t3}
got matrix of size (3, 2)
computing minors...
got GCD 1/(t2*t3)...


1/(t2*t3)

In [6]:
get_poly_wolfram([('v', [[2],[3,2]]),
 ('v', [[1,2],[1]])])

found: 3 segments and 3 edges
the homomorphism from pi_1(X) into an abelian group generated by edges is:
{x1: t1, x2: t2, x3: t3}
got matrix of size (3, 2)
computing minors...
got GCD t3^(-1)...


1/t3